In [1]:
import os
import glob
import json

import cv2
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

import torch
import torch.optim as optim
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms.functional as TF
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

### 데이터셋 만들기

In [2]:
classes = (
    'top', 'blouse', 't-shirt', 'Knitted fabri', 'shirt', 'bra top', 
    'hood', 'blue jeans', 'pants', 'skirt', 'leggings', 'jogger pants', 
    'coat', 'jacket', 'jumper', 'padding jacket', 'best', 'kadigan', 
    'zip up', 'dress', 'jumpsuit')


class FashionDataset(Dataset):
    def __init__(self, path, transforms=None):
        self.coco = COCO(path)
        self.image_ids = list(self.coco.imgToAnns.keys())
        self.transforms = transforms

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        file_name = self.coco.loadImgs(image_id)[0]['file_name']
        file_name = f'/content/drive/MyDrive/2021-K_fashion/train_new_all/{file_name}'
        image = Image.open(file_name).convert('RGB')

        annot_ids = self.coco.getAnnIds(imgIds=image_id)
        annots = [x for x in self.coco.loadAnns(annot_ids) if x['image_id'] == image_id]
        
        boxes = np.array([annot['bbox'] for annot in annots], dtype=np.float32)
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]

        labels = np.array([annot['category_id'] for annot in annots], dtype=np.int64)
        masks = np.array([self.coco.annToMask(annot) for annot in annots], dtype=np.uint8)

        area = np.array([annot['area'] for annot in annots], dtype=np.float32)
        iscrowd = np.array([annot['iscrowd'] for annot in annots], dtype=np.uint8)

        target = {
            'boxes': boxes,
            'masks': masks,
            'labels': labels,
            'area': area,
            'iscrowd': iscrowd,
            }
        
        if self.transforms is not None:
            image, target = self.transforms(image, target)
            
        target['boxes'] = torch.as_tensor(target['boxes'], dtype=torch.float32)
        target['masks'] = torch.as_tensor(target['masks'], dtype=torch.uint8)
        target['labels'] = torch.as_tensor(target['labels'], dtype=torch.int64)
        target['area'] = torch.as_tensor(target['area'], dtype=torch.float32)
        target['iscrowd'] = torch.as_tensor(target['iscrowd'], dtype=torch.uint8)            

        return image, target

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 데이터 어그멘테이션 만들기

In [4]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for transform in self.transforms:
            image, target = transform(
                image, target)

        return image, target


class Resize:
    def __init__(self, size, interpolation=Image.BILINEAR):
        self.size = size
        self.interpolation = interpolation

    def __call__(self, image, target):
        w, h = image.size
        image = image.resize(self.size)

        _masks = target['masks'].copy()
        masks = np.zeros((_masks.shape[0], self.size[0], self.size[1]))
        
        for i, v in enumerate(_masks):
            v = Image.fromarray(v).resize(self.size, resample=Image.BILINEAR)
            masks[i] = np.array(v, dtype=np.uint8)

        target['masks'] = masks
        target['boxes'][:, [0, 2]] *= self.size[0] / w
        target['boxes'][:, [1, 3]] *= self.size[1] / h
        
        return image, target
        

class ToTensor:
    def __call__(self, image, target):
        image = TF.to_tensor(image)
        
        return image, target

### Mask R-CNN 모형 만들기

In [6]:
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, len(classes)+1)
#model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
hidden_layer = 256
model.roi_heads.mask_predictor = MaskRCNNPredictor(
    in_features_mask, hidden_layer, len(classes)+1)

### 모형 학습하기

In [7]:
batch_size = 1
lr = 1e-3
max_size = 800
num_workers = 2
num_epochs = 1
device = 'cuda:0'

transforms_train = Compose([
    Resize((max_size, max_size)),
    ToTensor()])


def collate_fn(batch):
    return tuple(zip(*batch))


#dataset = FashionDataset('/content/drive/MyDrive/2021-K_fashion/train.json', mode='train', transforms=transforms_train)
dataset = FashionDataset('/content/drive/MyDrive/2021-K_fashion/train.json', transforms=transforms_train)
train_loader = DataLoader(
    dataset, batch_size=batch_size, shuffle=False, 
    num_workers=num_workers, collate_fn=collate_fn)


model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.Adam(
    params, lr=lr, weight_decay=1e-5)


loading annotations into memory...
Done (t=1.96s)
creating index...
index created!


In [9]:
def train_fn():
    model.train()
    for epoch in range(num_epochs):
        for i, (images, targets) in enumerate(train_loader):
            optimizer.zero_grad()
            images = [image.to(device) for image in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            try :
              losses = model(images, targets) 
              loss = sum(loss for loss in losses.values())
              print(
                  f"{epoch}, {i}, C: {losses['loss_classifier'].item():.5f}, M: {losses['loss_mask'].item():.5f}, "\
                  f"B: {losses['loss_box_reg'].item():.5f}, O: {losses['loss_objectness'].item():.5f}, T: {loss.item():.5f}")
              loss.backward()
              optimizer.step()
            except :
              print("예외 발생 : ", i)
              pass
                          

In [ ]:
train_fn()

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
0, 67598, C: 0.06526, M: 0.70499, B: 0.03486, O: 0.08831, T: 0.89464
0, 67599, C: 0.14370, M: 0.67261, B: 0.09181, O: 0.10115, T: 1.01429
0, 67600, C: 0.08660, M: 0.70905, B: 0.04313, O: 0.07960, T: 0.92165
0, 67601, C: 0.09847, M: 0.66635, B: 0.03994, O: 0.10242, T: 0.90843
0, 67602, C: 0.15256, M: 0.70046, B: 0.10648, O: 0.19876, T: 1.19182
0, 67603, C: 0.05707, M: 0.56282, B: 0.03081, O: 0.07211, T: 0.72527
0, 67604, C: 0.05641, M: 0.62005, B: 0.03666, O: 0.10680, T: 0.82107
0, 67605, C: 0.11135, M: 0.55914, B: 0.04384, O: 0.09810, T: 0.81287
0, 67606, C: 0.10751, M: 0.67685, B: 0.05714, O: 0.07552, T: 0.91893
0, 67607, C: 0.08870, M: 0.73736, B: 0.04858, O: 0.10830, T: 0.98371
0, 67608, C: 0.05513, M: 0.81869, B: 0.02551, O: 0.06909, T: 0.97148
0, 67609, C: 0.13996, M: 0.66972, B: 0.09835, O: 0.14363, T: 1.07118
0, 67610, C: 0.14326, M: 0.64794, B: 0.07384, O: 0.09973, T: 0.97546
0, 67611, C: 0.12675, M: 0.59111, B: 0.06982, O: 0.08823, T: 0.8804

In [1]:
!pip install pyyaml h5py 

In [3]:
model

NameError: ignored

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.